In [ ]:
import keras.layers as KL
from keras.models import Model
import keras.backend as K
import tensorflow as tf
import numpy as np
from model import FasterRcnn

import detection_target_fixed
import keras

In [ ]:
from utils import shapeData as dataSet
from config import Config

config = Config()
dataset = dataSet([64,64], config=config)

In [ ]:
model = FasterRcnn('inference','all',config)

In [ ]:
model.loadWeights("model_weights_all.h5")

In [ ]:
def data_Gen(dataset, num_batch, batch_size, config):
    for _ in range(num_batch):
        images = []
        bboxes = []
        class_ids = []
        rpn_matchs = []
        rpn_bboxes = []
        active_ids = []
        for i in range(batch_size):
            image, bbox, class_id, active_id, rpn_match, rpn_bbox, _ = data = dataset.load_data()
            pad_num = config.max_gt_obj - bbox.shape[0]
            pad_box = np.zeros((pad_num, 4))
            pad_ids = np.zeros((pad_num, 1))
            bbox = np.concatenate([bbox, pad_box], axis=0)
            class_id = np.concatenate([class_id, pad_ids], axis=0)
        
            images.append(image)
            bboxes.append(bbox)
            class_ids.append(class_id)
            rpn_matchs.append(rpn_match)
            rpn_bboxes.append(rpn_bbox)
            active_ids.append(active_id)
        images = np.concatenate(images, 0).reshape(batch_size, config.image_size[0],config.image_size[1] , 3)
        bboxes = np.concatenate(bboxes, 0).reshape(batch_size, -1 , 4)
        class_ids = np.concatenate(class_ids, 0).reshape(batch_size, -1 )
        active_ids = np.concatenate(active_ids, 0).reshape(batch_size, -1 )
        rpn_matchs = np.concatenate(rpn_matchs, 0).reshape(batch_size, -1 , 1)
        rpn_bboxes = np.concatenate(rpn_bboxes, 0).reshape(batch_size, -1 , 4)
        rpn_bboxes = np.concatenate(rpn_bboxes, 0).reshape(batch_size, -1 , 4)
        yield [images, bboxes, class_ids, active_ids, rpn_matchs, rpn_bboxes],[]

In [ ]:
dataGen = data_Gen(dataset, 35000, 20, config)

In [ ]:
test_data = next(dataGen)

In [ ]:
out = model.inference(test_data[0][0])

In [ ]:
out

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
%matplotlib inline

ix = random.sample(range(20), 1)[0]

image = test_data[0][0][ix]

boxes_result = out[ix][:,:4] * 64
id_result = out[ix][:,4]

plt.imshow(image)
Axs = plt.gca()

pos_idxs = np.where(id_result > 0)[0]

for i in range(pos_idxs.shape[0]):
    id_ = pos_idxs[i]
    box = boxes_result[id_]
    rec = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], 
                           edgecolor="r", facecolor="none")
    Axs.add_patch(rec)

In [ ]:
np.where(pos_idxs)[0]

In [ ]:
#rpn_class, rpn_prob, rpn_bbox, proposals, mrcnn_class_logits, mrcnn_class, mrcnn_bbox

proposals_model = out[3]
probs = out[5]
deltas_infe = out[-1]

print(proposals_model.shape)
print(probs.shape)
print(deltas_infe.shape)

In [ ]:
import random

sess = tf.Session()

ix = random.sample(range(20), 1)[0]
images = test_data[0][0]
image_ = images[ix]
rois_ = proposals_model[ix]
probs_ = probs[ix]
deltas = deltas_infe[ix]


def refine_detections(rois, probs, deltas):
    argMax_probs = tf.argmax(probs, axis=1)
    max_probs = tf.reduce_max(probs, axis=1)
    print("max_probs",sess.run(max_probs))
    keep_idxs = tf.where(max_probs > 0.9)[:,0]
    print("keep_idxs",sess.run(keep_idxs))
    idx_y = tf.cast(np.arange(16), tf.int32)
    idx_x = tf.cast(argMax_probs, tf.int32)
    idxs = tf.stack([idx_y, idx_x],axis=1)
    deltas_keep = tf.gather_nd(deltas, idxs)
    refined_rois = anchor_refinement(tf.cast(rois, tf.float32),
                                 tf.cast(deltas_keep * config.RPN_BBOX_STD_DEV, tf.float32))
    rois_ready = tf.gather(refined_rois, keep_idxs)
    class_ids = tf.gather(argMax_probs, keep_idxs)
    class_ids = tf.to_float(class_ids)[..., tf.newaxis]
    detections = tf.concat([rois_ready, class_ids], axis=1)
    gap = tf.maximum(8 - tf.shape(detections)[0],0)
    detections = tf.pad(detections, [(0, gap), (0, 0)], "CONSTANT")
    return detections

detections_ = refine_detections(rois_, probs_, deltas)

In [ ]:
max_probs = tf.reduce_max(probs_, axis=1)
keep_idxs = tf.where(max_probs > 0.1)[:,0]
print(sess.run(max_probs))
sess.run(keep_idxs)

In [ ]:
sess.run(detections_)

In [ ]:
proposal_ = proposals_model[ix] *64
image_ = images[ix]
probs_ = probs[ix]

In [ ]:
probs_.shape

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

print(np.argmax(probs_, axis=1))

plt.imshow(image_)
Axs = plt.gca()

for i in range(proposal_.shape[0]):
    box = proposal_[i]
    rec = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], 
                           edgecolor="r", facecolor="none")
    Axs.add_patch(rec)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
def anchor_refinement(boxes, deltas):
    boxes = tf.cast(boxes, tf.float32)
    h = boxes[:, 2] - boxes[:, 0]
    w = boxes[:, 3] - boxes[:, 1]
    center_y = boxes[:, 0] + h / 2
    center_x = boxes[:, 1] + w / 2

    center_y += deltas[:, 0] * h
    center_x += deltas[:, 1] * w
    h *= tf.exp(deltas[:, 2])
    w *= tf.exp(deltas[:, 3])
    
    y1 = center_y - h / 2
    x1 = center_x - w / 2
    y2 = center_y + h / 2
    x2 = center_x + w / 2
    boxes = tf.stack([y1, x1, y2, x2], axis=1)
    return boxes

In [ ]:
deltas_infe = out[-1][ix]
deltas_infe.shape

In [ ]:
out[-1].shape

In [ ]:
ixs = np.argmax(probs_, axis=1)

deltas = deltas_infe[np.arange(16), ixs, :]

In [ ]:
refined_rois = anchor_refinement(tf.cast(proposal_ / 64, tf.float32),
                                 tf.cast(deltas * config.RPN_BBOX_STD_DEV, tf.float32))

In [ ]:
refined_rois_ = sess.run(refined_rois) * 64

In [ ]:
positive_ixs = np.where(ixs > 0.4)[0]

positive_ixs

In [ ]:
plt.imshow(image_)
Axs = plt.gca()

for i in range(positive_ixs.shape[0]):
    box = refined_rois_[positive_ixs[i]]
    rec = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], 
                           edgecolor="b", facecolor="none")
    Axs.add_patch(rec)